In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


### English Cities Latitude and Longitude Values
Extract data sourced from https://www.townscountiespostcodes.co.uk/cities-in-england/ and https://www.latlong.net/category/cities-235-15.html plus some google searches to create a csv with 42 English cities coordinates data, excluding those have missing data.

In [2]:
# Create a df of England cities coordinates from a csv file
df = pd.read_csv('c:/Users/ecoms/Documents/IBM_Cert_DS/C9_Capstone/FinalProject/CityCoor.csv')
df.head()

,City,Latitude,Longitude
0,Bath,51.380001,-2.360000
1,Birmingham,52.489471,-1.898575
2,Bradford,53.799999,-1.750000
3,Bristol,51.454514,-2.587910
4,Cambridge,52.205276,0.119167


In [3]:
df.shape

(42, 3)

### Foursquare Nearby Venues Data Extraction

In [4]:
# Define Foursquare credentials and version
CLIENT_ID = 'removed - please insert your own'
CLIENT_SECRET = 'removed - please insert your own'
VERSION = '20200825' # Foursquare API version

In [5]:
# Get the City latitude and longitude values
city_latitude = df.loc[0, 'Latitude'] # city latitude value
city_longitude = df.loc[0, 'Longitude'] # city longitude value

city_name = df.loc[0, 'City'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(city_name, 
                                                               city_latitude, 
                                                               city_longitude))

Latitude and longitude values of Bath are 51.380001, -2.36.


In [6]:
# Create URL to extract list of venue, setting limit to 100 venues and within 500m radius of each city

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    city_latitude, 
    city_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=KPPGAX0UYGHHOCR54CDBIFI55BRPRTNKX0CFKESSXS0XUGMN&client_secret=VR24VX4ZQTSU1Q2BU3YMA1VZTYT0W1ANELRLCOLJVSUCZ5UD&v=20200825&ll=51.380001,-2.36&radius=500&limit=100'

In [7]:
# Get request and examine the results
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f4ad8242df029194ae5bc3b'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Bath',
  'headerFullLocation': 'Bath',
  'headerLocationGranularity': 'city',
  'totalResults': 54,
  'suggestedBounds': {'ne': {'lat': 51.3845010045, 'lng': -2.3528036760623072},
   'sw': {'lat': 51.3755009955, 'lng': -2.3671963239376925}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b6d7961f964a520db762ce3',
       'name': 'The Roman Baths',
       'location': {'address': 'Stall St',
        'lat': 51.38136993230506,
        'lng': -2.3596808928295556,
        'labeledLatLngs': [{'label': 'display',
          'lat': 51.38136993230506,
          'lng': -2.359680892

In [8]:
# Borrow the get_category_type function from the Foursquare lab

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [9]:
# Clean the json and structure it into a pandas dataframe.
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,The Roman Baths,Historic Site,51.381370,-2.359681
1,Acorn Vegetarian Kitchen,Vegetarian / Vegan Restaurant,51.380800,-2.358273
2,The Gainsborough Bath Spa Hotel,Hotel,51.380222,-2.361058
3,Society Cafe,Coffee Shop,51.381137,-2.362948
4,Ben's Cookies,Bakery,51.382056,-2.360164


In [10]:
# Find out how many venues were returned by Foursquare
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

54 venues were returned by Foursquare.


In [11]:
# Borrow the function to get nearby venues for all English cities

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 
                  'City Latitude', 
                  'City Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [12]:
# Create a new dataframe for all English cities venues
EngCities_venues = getNearbyVenues(names=df['City'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Bath
Birmingham
Bradford
Bristol
Cambridge
Canterbury
Carlisle
Chelmsford
Chester
Chichester
Coventry
Derby
Durham
Exeter
Gloucester
Hereford
Hull
Lancaster
Leeds
Leicester
Lichfield
Lincoln
Liverpool
London
Newcastle upon Tyne
Norwich
Nottingham
Oxford
Peterborough
Plymouth
Portsmouth
Preston
Sheffield
Southampton
St Albans
Stoke-on-Trent
Sunderland
Wakefield
Winchester
Wolverhampton
Worcester
York


In [13]:
# Check the resulting dataframe
print(EngCities_venues.shape)
EngCities_venues.head()

(2183, 7)


,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bath,51.380001,-2.36,The Roman Baths,51.381370,-2.359681,Historic Site
1,Bath,51.380001,-2.36,Acorn Vegetarian Kitchen,51.380800,-2.358273,Vegetarian / Vegan Restaurant
2,Bath,51.380001,-2.36,The Gainsborough Bath Spa Hotel,51.380222,-2.361058,Hotel
3,Bath,51.380001,-2.36,Society Cafe,51.381137,-2.362948,Coffee Shop
4,Bath,51.380001,-2.36,Ben's Cookies,51.382056,-2.360164,Bakery


In [14]:
# Check how many venues returned for each city
EngCities_venues.groupby('City').count()

,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
City,,,,,,
Bath,54,54,54,54,54,54
Birmingham,11,11,11,11,11,11
Bradford,21,21,21,21,21,21
Bristol,100,100,100,100,100,100
Cambridge,88,88,88,88,88,88
Canterbury,100,100,100,100,100,100
Carlisle,8,8,8,8,8,8
Chelmsford,32,32,32,32,32,32
Chester,52,52,52,52,52,52


In [15]:
# Find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(EngCities_venues['Venue Category'].unique())))

There are 207 uniques categories.


### Analyze Each City Using One Hot Encoding

In [16]:
# one hot encoding
EngCities_onehot = pd.get_dummies(EngCities_venues[['Venue Category']], prefix="", prefix_sep="")

# add city column back to dataframe
EngCities_onehot['City'] = EngCities_venues['City'] 

# move citycolumn to the first column
fixed_columns = [EngCities_onehot.columns[-1]] + list(EngCities_onehot.columns[:-1])
EngCities_onehot = EngCities_onehot[fixed_columns]

EngCities_onehot.head()

,City,Accessories Store,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Auto Garage,BBQ Joint,...,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Wine Bar,Wine Shop,Women's Store
0,Bath,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Bath,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,Bath,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Bath,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Bath,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
# Examine new dataframe size
EngCities_onehot.shape

(2183, 208)

In [18]:
# Group rows by city and by taking the mean of the frequency of occurrence of each category
EngCities_grouped = EngCities_onehot.groupby('City').mean().reset_index()
EngCities_grouped

,City,Accessories Store,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Auto Garage,BBQ Joint,...,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Wine Bar,Wine Shop,Women's Store
0,Bath,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.018519,0.000000,0.000000,0.018519,0.000000,0.000,0.000000,0.000000,0.000000
1,Birmingham,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000
2,Bradford,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.000,0.000000,0.000000,0.000000
3,Bristol,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.000000,...,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000
4,Cambridge,0.000000,0.011364,0.000000,0.000000,0.000000,0.000000,0.011364,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000
5,Canterbury,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.010000,0.000000,0.020000,...,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.010000
6,Carlisle,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000
7,Chelmsford,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000
8,Chester,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.019231,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000
9,Chichester,0.000000,0.000000,0.000000,0.000000,0.024390,0.000000,0.024390,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.024390,0.000000,0.024390


### Show Each City's Top 5 Most Common Venues

In [19]:
# Print each city along with the top 5 most common venues
num_top_venues = 5

for hood in EngCities_grouped['City']:
    print("----"+hood+"----")
    temp = EngCities_grouped[EngCities_grouped['City'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bath----
          venue  freq
0           Pub  0.11
1   Coffee Shop  0.09
2          Café  0.07
3         Hotel  0.06
4  Cocktail Bar  0.06


----Birmingham----
               venue  freq
0                Pub  0.18
1  Indian Restaurant  0.09
2       Climbing Gym  0.09
3            Brewery  0.09
4        Escape Room  0.09


----Bradford----
                 venue  freq
0       Clothing Store  0.19
1                Hotel  0.10
2  Sporting Goods Shop  0.05
3        Grocery Store  0.05
4                  Pub  0.05


----Bristol----
                venue  freq
0         Coffee Shop  0.08
1                 Pub  0.08
2                Café  0.08
3  Italian Restaurant  0.06
4                 Bar  0.05


----Cambridge----
            venue  freq
0             Pub  0.10
1     Coffee Shop  0.08
2  Clothing Store  0.07
3            Café  0.06
4    Burger Joint  0.05


----Canterbury----
                venue  freq
0                 Pub  0.10
1         Coffee Shop  0.10
2                Café  0

### Show Each City Top 10 Most Common Venues

In [20]:
# Create a function to return most common venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [21]:
# Create a new dataframe and display the top 10 venues for each city

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['City']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
cities_venues_sorted = pd.DataFrame(columns=columns)
cities_venues_sorted['City'] = EngCities_grouped['City']

for ind in np.arange(EngCities_grouped.shape[0]):
    cities_venues_sorted.iloc[ind, 1:] = return_most_common_venues(EngCities_grouped.iloc[ind, :], num_top_venues)

cities_venues_sorted.head()

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bath,Pub,Coffee Shop,Café,Cocktail Bar,Hotel,Pizza Place,Tea Room,Theater,Bar,French Restaurant
1,Birmingham,Pub,Pizza Place,Brewery,Sandwich Place,Bar,Lebanese Restaurant,Climbing Gym,Music Venue,Indian Restaurant,Escape Room
2,Bradford,Clothing Store,Hotel,Coffee Shop,Bakery,Shoe Store,Pharmacy,Sporting Goods Shop,Discount Store,Sandwich Place,Supermarket
3,Bristol,Café,Pub,Coffee Shop,Italian Restaurant,Bar,Burger Joint,Bookstore,Clothing Store,Steakhouse,Music Venue
4,Cambridge,Pub,Coffee Shop,Clothing Store,Café,Burger Joint,English Restaurant,Sushi Restaurant,Pharmacy,Science Museum,Bookstore


In [22]:
# Left Join dataframes of city data with top venue and coordinates
cities_venues_sorted = cities_venues_sorted.merge(df, on='City', how='left')
cities_venues_sorted.head()

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Latitude,Longitude
0,Bath,Pub,Coffee Shop,Café,Cocktail Bar,Hotel,Pizza Place,Tea Room,Theater,Bar,French Restaurant,51.380001,-2.360000
1,Birmingham,Pub,Pizza Place,Brewery,Sandwich Place,Bar,Lebanese Restaurant,Climbing Gym,Music Venue,Indian Restaurant,Escape Room,52.489471,-1.898575
2,Bradford,Clothing Store,Hotel,Coffee Shop,Bakery,Shoe Store,Pharmacy,Sporting Goods Shop,Discount Store,Sandwich Place,Supermarket,53.799999,-1.750000
3,Bristol,Café,Pub,Coffee Shop,Italian Restaurant,Bar,Burger Joint,Bookstore,Clothing Store,Steakhouse,Music Venue,51.454514,-2.587910
4,Cambridge,Pub,Coffee Shop,Clothing Store,Café,Burger Joint,English Restaurant,Sushi Restaurant,Pharmacy,Science Museum,Bookstore,52.205276,0.119167


### Property Price
Create a new dataframe with house price index in January 2020 sourced from https://www.gov.uk/government/publications/uk-house-price-index-england-january-2020/uk-house-price-index-england-january-2020 

In [23]:
# Create a df of UK house price index from a csv file
df_p = pd.read_csv('c:/Users/ecoms/Documents/IBM_Cert_DS/C9_Capstone/FinalProject/Property.csv', encoding= 'unicode_escape')
df_p.head()

,Local authorities,January 2020,January 2019,Difference
0,Adur,£325359,£315376,3.2%
1,Allerdale,£154319,£148231,4.1%
2,Amber Valley,£179734,£170846,5.2%
3,Arun,£283048,£281179,0.7%
4,Ashfield,£144404,£139432,3.6%


In [24]:
# Rename Local authorities column to City; and January 2020 to Property_Jan_2020
df_p.rename(columns={'Local authorities': 'City', 'January 2020': 'Property_Jan_2020'}, inplace=True)

# Drop January 2019 and Difference columns
df_p = df_p.drop(['January 2019', 'Difference'], axis=1)

df_p.head()

,City,Property_Jan_2020
0,Adur,£325359
1,Allerdale,£154319
2,Amber Valley,£179734
3,Arun,£283048
4,Ashfield,£144404


In [25]:
# Replace value to match with cities_venues_sorted dataframe
df_p = df_p.replace(['Bath and North East Somerset','City of Bristol', 'Cheshire West and Chester', 'City of Derby', 'County Durham', 'Herefordshire', 'City of Kingston upon Hull', 'City of London', 'City of Nottingham', 'City of Peterborough', 'City of Plymouth'],['Bath','Bristol', 'Chester', 'Derby', 'Durham', 'Hereford', 'Hull', 'London', 'Nottingham', 'Peterborough', 'Plymouth'])
df_p

,City,Property_Jan_2020
0,Adur,£325359
1,Allerdale,£154319
2,Amber Valley,£179734
3,Arun,£283048
4,Ashfield,£144404
...,...,...
348,Wycombe,£391615
349,Wyre,£160109
350,Wyre Forest,£203225
351,York,£255955


In [26]:
# Remove £ sign and convert value in Property_Jan_2020 column from object to integer
df_p['Property_Jan_2020']  = df_p['Property_Jan_2020'].str.lstrip('£').astype('int')
df_p.head()

,City,Property_Jan_2020
0,Adur,325359
1,Allerdale,154319
2,Amber Valley,179734
3,Arun,283048
4,Ashfield,144404


### Crime data
Create a new dataframe with crime statistics based on year ending March 2020 sourced from the National Office of Statisitcs to pull crime per 1,000 population/household. Microsoft Excel vlookup is used to extract data that match with the English cities list to create a CityCrime csv file. https://www.ons.gov.uk/peoplepopulationandcommunity/crimeandjustice/datasets/recordedcrimedatabycommunitysafetypartnershiparea 

In [27]:
# Create a df of Crime rate from a csv file
df_c = pd.read_csv('c:/Users/ecoms/Documents/IBM_Cert_DS/C9_Capstone/FinalProject/CityCrime.csv')
df_c.head()

,City,Total crime,Violent crime,Theft offences
0,Bath,65.1,20.5,23.5
1,Birmingham,100.5,35.7,36.2
2,Bradford,136.1,53.2,38.8
3,Brighton and Hove,100.5,31.4,37.8
4,Bristol,114.1,34.7,40.3


In [28]:
# Convert columns from object to float.
df_c['Total crime'] = pd.to_numeric(df_c['Total crime'], errors='coerce')
df_c['Violent crime'] = pd.to_numeric(df_c['Violent crime'], errors='coerce')
df_c['Theft offences'] = pd.to_numeric(df_c['Theft offences'], errors='coerce')
df_c.dtypes

City               object
Total crime       float64
Violent crime     float64
Theft offences    float64
dtype: object

In [29]:
# Left Join dataframe of Property and Crime data to create a new ONS1 dataframe
df_ons1 = df_c.merge(df_p, on='City', how='left')
df_ons1.head()

,City,Total crime,Violent crime,Theft offences,Property_Jan_2020
0,Bath,65.1,20.5,23.5,330975.0
1,Birmingham,100.5,35.7,36.2,189161.0
2,Bradford,136.1,53.2,38.8,133763.0
3,Brighton and Hove,100.5,31.4,37.8,368224.0
4,Bristol,114.1,34.7,40.3,285296.0


### Employment data
Create a new dataframe with January to December 2019 employment rate data sourced from nomis that provides official labour market. Microsoft Excel vlookup is used to extract data that match with the English cities list to create an Employment csv file. https://www.nomisweb.co.uk/query/construct/submit.asp?menuopt=201&subcomp= 

In [30]:
# Create a df of employment rate from a csv file
df_em = pd.read_csv('c:/Users/ecoms/Documents/IBM_Cert_DS/C9_Capstone/FinalProject/Employment.csv')
df_em.head()

,City,Employment rate
0,Bath,0.731638
1,Birmingham,0.640017
2,Bradford,0.627727
3,Brighton and Hove,0.769443
4,Bristol,0.778697


In [31]:
df_em.dtypes

City                object
Employment rate    float64
dtype: object

In [32]:
# Left Join dataframe of Property, Crime, Employment data to create a new ONS2 dataframe
df_ons2 = df_ons1.merge(df_em, on='City', how='left')
df_ons2.head()

,City,Total crime,Violent crime,Theft offences,Property_Jan_2020,Employment rate
0,Bath,65.1,20.5,23.5,330975.0,0.731638
1,Birmingham,100.5,35.7,36.2,189161.0,0.640017
2,Bradford,136.1,53.2,38.8,133763.0,0.627727
3,Brighton and Hove,100.5,31.4,37.8,368224.0,0.769443
4,Bristol,114.1,34.7,40.3,285296.0,0.778697


### Education data
Create two dataframes with 2018-2019 Key Stage 2 and Key Stage 4 performance data to cover primary education and secondary education sourced from the Department of Education. Using percentgage of pupils meeting expected standard to compare primary school performance and attainment 8 score to compare secondary school performance. https://www.compare-school-performance.service.gov.uk/download-data

In [33]:
# Create a df of education KS2 results from a csv file
df_ed2 = pd.read_csv('c:/Users/ecoms/Documents/IBM_Cert_DS/C9_Capstone/FinalProject/ks2_exp.csv')
df_ed2.head()

,City,MeetExp
0,Market Harborough,0.96
1,Grantham,0.65
2,Melton Mowbray,0.86
3,Loughborough,0.92
4,Coalville,0.49


In [34]:
df_ed2.shape

(15896, 2)

In [35]:
# Check the shape of this dataframe
df_ed2.dtypes

City       object
MeetExp    object
dtype: object

In [36]:
# Convert columns from object to float.
df_ed2['MeetExp'] = pd.to_numeric(df_ed2['MeetExp'], errors='coerce')
df_ed2.dtypes

City        object
MeetExp    float64
dtype: object

In [37]:
# Group the data by 'City', aggregate and calculate the mean for each column
gp_ks2 = df_ed2.groupby('City').agg({'MeetExp':['mean']})
gp_ks2 = gp_ks2.reset_index()
gp_ks2.head()

,City,MeetExp
,,mean
0,,0.65915
1,Abberley,0.85000
2,Abbots Langley,0.64000
3,Abingdon,0.66375
4,Accrington,0.56920


In [38]:
gp_ks2.shape

(1224, 2)

In [39]:
# Left Join dataframe of Property, Crime, Employment, KS2 data to create a new ONS3 dataframe
df_ons3 = df_ons2.merge(gp_ks2, on='City', how='left')
df_ons3.head()

C:\Users\ecoms\anaconda3\lib\site-packages\pandas\core\reshape\merge.py:618: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
C:\Users\ecoms\anaconda3\lib\site-packages\pandas\core\generic.py:3936: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


,City,Total crime,Violent crime,Theft offences,Property_Jan_2020,Employment rate,"(MeetExp, mean)"
0,Bath,65.1,20.5,23.5,330975.0,0.731638,0.692414
1,Birmingham,100.5,35.7,36.2,189161.0,0.640017,0.635331
2,Bradford,136.1,53.2,38.8,133763.0,0.627727,0.617545
3,Brighton and Hove,100.5,31.4,37.8,368224.0,0.769443,0.840000
4,Bristol,114.1,34.7,40.3,285296.0,0.778697,0.658955


In [40]:
# Create a df of education KS4 results from a csv file
df_ed4 = pd.read_csv('c:/Users/ecoms/Documents/IBM_Cert_DS/C9_Capstone/FinalProject/ks4_at8.csv')
df_ed4.head()

,City,ATT8SCR
0,London,57.9
1,London,20.0
2,London,42.4
3,London,58.7
4,London,16.4


In [41]:
# Check the shape of this dataframe
df_ed4.dtypes

City        object
ATT8SCR    float64
dtype: object

In [42]:
df_ed4.shape

(3952, 2)

In [43]:
# Group the data by 'City', aggregate and calculate the mean for each column
gp_ks4 = df_ed4.groupby('City').agg({'ATT8SCR':['mean']})
gp_ks4 = gp_ks4.reset_index()
gp_ks4.shape

(903, 2)

In [44]:
# Left Join dataframe of Property, Crime, Employment, KS2 and KS4 data to create a new ONS4 dataframe
df_ons4 = df_ons3.merge(gp_ks4, on='City', how='left')
df_ons4.head()

C:\Users\ecoms\anaconda3\lib\site-packages\pandas\core\reshape\merge.py:618: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
C:\Users\ecoms\anaconda3\lib\site-packages\pandas\core\generic.py:3936: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


,City,Total crime,Violent crime,Theft offences,Property_Jan_2020,Employment rate,"(MeetExp, mean)","(ATT8SCR, mean)"
0,Bath,65.1,20.5,23.5,330975.0,0.731638,0.692414,39.353846
1,Birmingham,100.5,35.7,36.2,189161.0,0.640017,0.635331,45.800971
2,Bradford,136.1,53.2,38.8,133763.0,0.627727,0.617545,41.223529
3,Brighton and Hove,100.5,31.4,37.8,368224.0,0.769443,0.840000,NaN
4,Bristol,114.1,34.7,40.3,285296.0,0.778697,0.658955,43.990566


In [45]:
df_ons4.dropna(inplace=True)
df_ons4.shape

(42, 8)

In [46]:
df_ons4.dtypes

City                  object
Total crime          float64
Violent crime        float64
Theft offences       float64
Property_Jan_2020    float64
Employment rate      float64
(MeetExp, mean)      float64
(ATT8SCR, mean)      float64
dtype: object

In [47]:
# Left Join dataframes of EngCities_grouped with the ONS table
df_city = EngCities_grouped.merge(df_ons4, on='City', how='left')
df_city.head()

,City,Accessories Store,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Auto Garage,BBQ Joint,...,Wine Bar,Wine Shop,Women's Store,Total crime,Violent crime,Theft offences,Property_Jan_2020,Employment rate,"(MeetExp, mean)","(ATT8SCR, mean)"
0,Bath,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,65.1,20.5,23.5,330975.0,0.731638,0.692414,39.353846
1,Birmingham,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,100.5,35.7,36.2,189161.0,0.640017,0.635331,45.800971
2,Bradford,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,136.1,53.2,38.8,133763.0,0.627727,0.617545,41.223529
3,Bristol,0.0,0.010000,0.0,0.0,0.0,0.0,0.020000,0.0,0.0,...,0.0,0.0,0.0,114.1,34.7,40.3,285296.0,0.778697,0.658955,43.990566
4,Cambridge,0.0,0.011364,0.0,0.0,0.0,0.0,0.011364,0.0,0.0,...,0.0,0.0,0.0,127.1,30.9,63.9,435174.0,0.809375,0.688630,48.104762


### Use K-means cluster to segment the city

In [48]:
# set number of clusters
kclusters = 5

df_city_clustering = df_city.drop('City', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_city_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 0, 4, 3, 2, 3, 4, 3, 0, 2])

Create a new dataframe that include the cluster and top 10 venues for each city.

In [49]:
# add clustering labels
cities_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

cities_merged = df_ons4

# merge cities_grouped with cities_data to add top 10 venues for each cities
cities_merged = cities_merged.join(cities_venues_sorted.set_index('City'), on='City')


cities_merged.head() # check the last columns!

,City,Total crime,Violent crime,Theft offences,Property_Jan_2020,Employment rate,"(MeetExp, mean)","(ATT8SCR, mean)",Cluster Labels,1st Most Common Venue,...,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Latitude,Longitude
0,Bath,65.1,20.5,23.5,330975.0,0.731638,0.692414,39.353846,3,Pub,...,Café,Cocktail Bar,Hotel,Pizza Place,Tea Room,Theater,Bar,French Restaurant,51.380001,-2.360000
1,Birmingham,100.5,35.7,36.2,189161.0,0.640017,0.635331,45.800971,0,Pub,...,Brewery,Sandwich Place,Bar,Lebanese Restaurant,Climbing Gym,Music Venue,Indian Restaurant,Escape Room,52.489471,-1.898575
2,Bradford,136.1,53.2,38.8,133763.0,0.627727,0.617545,41.223529,4,Clothing Store,...,Coffee Shop,Bakery,Shoe Store,Pharmacy,Sporting Goods Shop,Discount Store,Sandwich Place,Supermarket,53.799999,-1.750000
4,Bristol,114.1,34.7,40.3,285296.0,0.778697,0.658955,43.990566,3,Café,...,Coffee Shop,Italian Restaurant,Bar,Burger Joint,Bookstore,Clothing Store,Steakhouse,Music Venue,51.454514,-2.587910
5,Cambridge,127.1,30.9,63.9,435174.0,0.809375,0.688630,48.104762,2,Pub,...,Clothing Store,Café,Burger Joint,English Restaurant,Sushi Restaurant,Pharmacy,Science Museum,Bookstore,52.205276,0.119167


### Visualise the results

In [50]:
# create map
map_clusters = folium.Map(location=[52.408054,-1.510556], zoom_start=6)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(cities_merged['Latitude'], cities_merged['Longitude'], cities_merged['City'], cities_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters
Let's examine each clusters to see the characteristics.

In [51]:
# Drop Latitude and Longitude to display all data
cities_merged.drop(['Latitude', 'Longitude'], axis=1, inplace=True)
cities_merged.head()

,City,Total crime,Violent crime,Theft offences,Property_Jan_2020,Employment rate,"(MeetExp, mean)","(ATT8SCR, mean)",Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bath,65.1,20.5,23.5,330975.0,0.731638,0.692414,39.353846,3,Pub,Coffee Shop,Café,Cocktail Bar,Hotel,Pizza Place,Tea Room,Theater,Bar,French Restaurant
1,Birmingham,100.5,35.7,36.2,189161.0,0.640017,0.635331,45.800971,0,Pub,Pizza Place,Brewery,Sandwich Place,Bar,Lebanese Restaurant,Climbing Gym,Music Venue,Indian Restaurant,Escape Room
2,Bradford,136.1,53.2,38.8,133763.0,0.627727,0.617545,41.223529,4,Clothing Store,Hotel,Coffee Shop,Bakery,Shoe Store,Pharmacy,Sporting Goods Shop,Discount Store,Sandwich Place,Supermarket
4,Bristol,114.1,34.7,40.3,285296.0,0.778697,0.658955,43.990566,3,Café,Pub,Coffee Shop,Italian Restaurant,Bar,Burger Joint,Bookstore,Clothing Store,Steakhouse,Music Venue
5,Cambridge,127.1,30.9,63.9,435174.0,0.809375,0.688630,48.104762,2,Pub,Coffee Shop,Clothing Store,Café,Burger Joint,English Restaurant,Sushi Restaurant,Pharmacy,Science Museum,Bookstore


#### Cluster 1

In [52]:
cities_merged.loc[cities_merged['Cluster Labels'] == 0, cities_merged.columns[[0] + list(range(1, cities_merged.shape[1]))]]

,City,Total crime,Violent crime,Theft offences,Property_Jan_2020,Employment rate,"(MeetExp, mean)","(ATT8SCR, mean)",Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Birmingham,100.5,35.7,36.2,189161.0,0.640017,0.635331,45.800971,0,Pub,Pizza Place,Brewery,Sandwich Place,Bar,Lebanese Restaurant,Climbing Gym,Music Venue,Indian Restaurant,Escape Room
9,Chester,85.2,33.7,20.9,209431.0,0.800380,0.658000,46.454545,0,Pub,Hotel,Restaurant,Historic Site,Italian Restaurant,Bar,Café,Steakhouse,Coffee Shop,Brazilian Restaurant
11,Coventry,77.8,25.5,31.0,184305.0,0.732214,0.638191,40.855556,0,Pub,Coffee Shop,Café,Clothing Store,Bar,Gym / Fitness Center,Sandwich Place,Italian Restaurant,Chinese Restaurant,Grocery Store
16,Gloucester,104.2,37.1,36.0,210982.0,0.840471,0.646792,50.700000,0,Pub,Coffee Shop,Sandwich Place,Pharmacy,Bookstore,Clothing Store,Grocery Store,Café,Furniture / Home Store,Discount Store
20,Leeds,130.6,46.2,45.0,191578.0,0.712668,0.625380,43.167568,0,Coffee Shop,Café,Hotel,Bar,Pizza Place,Gym / Fitness Center,Bookstore,Music Venue,Pub,Steakhouse
21,Leicester,115.7,40.5,36.6,177556.0,0.720974,0.664823,44.181818,0,Coffee Shop,Pub,Bar,Indian Restaurant,Shopping Mall,Café,Italian Restaurant,Restaurant,History Museum,Nightclub
28,Norwich,135.8,48.8,40.0,202582.0,0.774290,0.642636,42.357143,0,Pub,Coffee Shop,Italian Restaurant,Café,Bar,Cocktail Bar,Indian Restaurant,Pizza Place,Restaurant,Burger Joint
31,Peterborough,109.8,36.5,38.2,199105.0,0.732090,0.609221,42.073684,0,Platform,Clothing Store,Coffee Shop,Pub,Burger Joint,Women's Store,Hotel,Sandwich Place,Restaurant,Pizza Place
32,Plymouth,87.8,36.5,22.4,176390.0,0.739620,0.640411,43.989474,0,Platform,Coffee Shop,Restaurant,Grocery Store,Pub,Student Center,Furniture / Home Store,Italian Restaurant,Smoothie Shop,Vegetarian / Vegan Restaurant
33,Portsmouth,118.4,46.3,35.1,212171.0,0.717864,0.577407,43.130000,0,Fast Food Restaurant,Pub,Sporting Goods Shop,Supermarket,Sandwich Place,Chinese Restaurant,Asian Restaurant,History Museum,Video Game Store,Warehouse Store


In [53]:
# Analyse Cluster 1 numeric data
cluster_1 = cities_merged.loc[cities_merged['Cluster Labels'] == 0, cities_merged.columns[[0] + list(range(1, cities_merged.shape[1]))]]
cluster_1.describe(exclude=[object])

,Total crime,Violent crime,Theft offences,Property_Jan_2020,Employment rate,"(MeetExp, mean)","(ATT8SCR, mean)",Cluster Labels
count,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.0
mean,104.830769,37.723077,34.223077,196070.384615,0.745918,0.641454,44.036201,0.0
std,20.688579,7.715045,7.542453,15003.178977,0.050055,0.027792,2.631869,0.0
min,70.200000,25.500000,20.900000,172357.000000,0.640017,0.577407,40.733333,0.0
25%,87.800000,33.700000,31.000000,184305.000000,0.720974,0.635331,42.357143,0.0
50%,104.200000,36.500000,36.200000,199105.000000,0.738864,0.642636,43.989474,0.0
75%,118.400000,46.200000,38.400000,210018.000000,0.774290,0.658000,44.539024,0.0
max,135.800000,48.800000,45.000000,213279.000000,0.840471,0.691111,50.700000,0.0


#### Cluster 2

In [54]:
cities_merged.loc[cities_merged['Cluster Labels'] == 1, cities_merged.columns[[0] + list(range(1, cities_merged.shape[1]))]]

,City,Total crime,Violent crime,Theft offences,Property_Jan_2020,Employment rate,"(MeetExp, mean)","(ATT8SCR, mean)",Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,London,101.6,24.8,50.5,775309.0,0.74506,0.721974,46.574713,1,Theater,Burger Joint,Coffee Shop,Pub,Hotel,Restaurant,Bookstore,Gelato Shop,Art Gallery,Japanese Restaurant


#### Cluster 3

In [55]:
cities_merged.loc[cities_merged['Cluster Labels'] == 2, cities_merged.columns[[0] + list(range(1, cities_merged.shape[1]))]]

,City,Total crime,Violent crime,Theft offences,Property_Jan_2020,Employment rate,"(MeetExp, mean)","(ATT8SCR, mean)",Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Cambridge,127.1,30.9,63.9,435174.0,0.809375,0.688630,48.104762,2,Pub,Coffee Shop,Clothing Store,Café,Burger Joint,English Restaurant,Sushi Restaurant,Pharmacy,Science Museum,Bookstore
10,Chichester,61.0,18.1,25.2,375252.0,0.874816,0.602800,43.700000,2,Pub,Italian Restaurant,Coffee Shop,Clothing Store,Park,Thai Restaurant,Café,Tea Room,French Restaurant,Department Store
30,Oxford,109.4,27.6,55.0,433918.0,0.794011,0.617674,42.600000,2,Coffee Shop,Pub,Café,Restaurant,Sandwich Place,Art Gallery,Bookstore,Hotel,Thai Restaurant,Breakfast Spot
40,St Albans,61.1,17.8,27.0,502294.0,0.790497,0.694286,46.376923,2,Coffee Shop,Pub,Sandwich Place,French Restaurant,Market,Dessert Shop,Café,Theater,Mediterranean Restaurant,Bar
47,Winchester,67.7,21.2,27.4,395768.0,0.781333,0.762857,47.280000,2,Pub,Bakery,Coffee Shop,Clothing Store,Café,Pizza Place,Department Store,Restaurant,Sandwich Place,Bookstore


In [56]:
# Analyse Cluster 3 numeric data
cluster_3 = cities_merged.loc[cities_merged['Cluster Labels'] == 2, cities_merged.columns[[0] + list(range(1, cities_merged.shape[1]))]]
cluster_3.describe(exclude=[object])

,Total crime,Violent crime,Theft offences,Property_Jan_2020,Employment rate,"(MeetExp, mean)","(ATT8SCR, mean)",Cluster Labels
count,5.00000,5.000000,5.00000,5.000000,5.000000,5.000000,5.000000,5.0
mean,85.26000,23.120000,39.70000,428481.200000,0.810006,0.673249,45.612337,2.0
std,30.87852,5.869157,18.32048,48544.265874,0.037614,0.064731,2.361624,0.0
min,61.00000,17.800000,25.20000,375252.000000,0.781333,0.602800,42.600000,2.0
25%,61.10000,18.100000,27.00000,395768.000000,0.790497,0.617674,43.700000,2.0
50%,67.70000,21.200000,27.40000,433918.000000,0.794011,0.688630,46.376923,2.0
75%,109.40000,27.600000,55.00000,435174.000000,0.809375,0.694286,47.280000,2.0
max,127.10000,30.900000,63.90000,502294.000000,0.874816,0.762857,48.104762,2.0


#### Cluster 4

In [57]:
cities_merged.loc[cities_merged['Cluster Labels'] == 3, cities_merged.columns[[0] + list(range(1, cities_merged.shape[1]))]]

,City,Total crime,Violent crime,Theft offences,Property_Jan_2020,Employment rate,"(MeetExp, mean)","(ATT8SCR, mean)",Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bath,65.1,20.5,23.5,330975.0,0.731638,0.692414,39.353846,3,Pub,Coffee Shop,Café,Cocktail Bar,Hotel,Pizza Place,Tea Room,Theater,Bar,French Restaurant
4,Bristol,114.1,34.7,40.3,285296.0,0.778697,0.658955,43.990566,3,Café,Pub,Coffee Shop,Italian Restaurant,Bar,Burger Joint,Bookstore,Clothing Store,Steakhouse,Music Venue
6,Canterbury,97.8,36.8,31.6,302100.0,0.731429,0.711034,44.020000,3,Pub,Coffee Shop,Café,Italian Restaurant,Bar,Gastropub,Burger Joint,Clothing Store,Plaza,Sandwich Place
8,Chelmsford,92.0,34.6,28.5,331973.0,0.815789,0.676327,54.772727,3,Pub,Italian Restaurant,Bar,Department Store,Burger Joint,Toy / Game Store,Steakhouse,Discount Store,Latin American Restaurant,Sushi Restaurant
15,Exeter,74.7,28.4,21.3,262342.0,0.773933,0.676000,44.990000,3,Pub,Café,Clothing Store,Tea Room,Gym / Fitness Center,Grocery Store,Pizza Place,Coffee Shop,Climbing Gym,Sandwich Place
17,Hereford,56.3,23.1,17.5,241217.0,0.843224,0.692381,45.530000,3,Pub,Clothing Store,Coffee Shop,Café,Bar,Burger Joint,Hotel,Bookstore,Department Store,Sandwich Place
22,Lichfield,57.0,21.2,21.6,259701.0,0.793831,0.712500,49.180000,3,Pub,Coffee Shop,Bar,Pharmacy,Café,Hotel,Grocery Store,Restaurant,Cocktail Bar,Discount Store
50,York,65.3,21.8,25.7,255955.0,0.770476,0.687097,46.160870,3,Pub,Café,Bar,Cocktail Bar,Italian Restaurant,Historic Site,Restaurant,Plaza,Coffee Shop,Tapas Restaurant


In [58]:
# Analyse Cluster 4 numeric data
cluster_4 = cities_merged.loc[cities_merged['Cluster Labels'] == 3, cities_merged.columns[[0] + list(range(1, cities_merged.shape[1]))]]
cluster_4.describe(exclude=[object])

,Total crime,Violent crime,Theft offences,Property_Jan_2020,Employment rate,"(MeetExp, mean)","(ATT8SCR, mean)",Cluster Labels
count,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.0
mean,77.787500,27.637500,26.250000,283694.875000,0.779877,0.688338,45.999751,3.0
std,21.184121,6.865428,7.187688,34873.284164,0.038353,0.018100,4.479630,0.0
min,56.300000,20.500000,17.500000,241217.000000,0.731429,0.658955,39.353846,3.0
25%,63.075000,21.650000,21.525000,258764.500000,0.760767,0.676245,44.012642,3.0
50%,70.000000,25.750000,24.600000,273819.000000,0.776315,0.689739,45.260000,3.0
75%,93.450000,34.625000,29.275000,309318.750000,0.799321,0.697069,46.915652,3.0
max,114.100000,36.800000,40.300000,331973.000000,0.843224,0.712500,54.772727,3.0


#### Cluster 5

In [59]:
cities_merged.loc[cities_merged['Cluster Labels'] == 4, cities_merged.columns[[0] + list(range(1, cities_merged.shape[1]))]]

,City,Total crime,Violent crime,Theft offences,Property_Jan_2020,Employment rate,"(MeetExp, mean)","(ATT8SCR, mean)",Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Bradford,136.1,53.2,38.8,133763.0,0.627727,0.617545,41.223529,4,Clothing Store,Hotel,Coffee Shop,Bakery,Shoe Store,Pharmacy,Sporting Goods Shop,Discount Store,Sandwich Place,Supermarket
7,Carlisle,103.3,41.9,22.5,136912.0,0.810924,0.661795,36.214286,4,Supermarket,Coffee Shop,Rental Car Location,Furniture / Home Store,Grocery Store,Sandwich Place,Indian Restaurant,Convenience Store,Diner,Event Space
12,Derby,111.0,43.0,33.9,159403.0,0.749548,0.628706,43.481818,4,Platform,Coffee Shop,Pub,Clothing Store,Indian Restaurant,Bookstore,Portuguese Restaurant,Restaurant,Department Store,Dessert Shop
13,Durham,97.8,39.6,27.7,100643.0,0.737291,0.708444,47.711111,4,Coffee Shop,Café,Pub,Italian Restaurant,Burger Joint,English Restaurant,Plaza,Bar,Cocktail Bar,Asian Restaurant
18,Hull,158.0,55.4,51.3,113565.0,0.717155,0.665632,41.962500,4,Coffee Shop,Café,Grocery Store,Gay Bar,Bookstore,Pub,Music Store,Women's Store,Park,Sandwich Place
19,Lancaster,82.9,31.5,27.4,151609.0,0.693723,0.652174,51.050000,4,Pub,Clothing Store,Sandwich Place,Coffee Shop,Hotel,Indian Restaurant,Bookstore,Deli / Bodega,Castle,Restaurant
23,Lincoln,149.6,50.7,48.3,156640.0,0.755014,0.613125,42.360000,4,Pub,Bar,Café,Hotel,Tea Room,Nightclub,History Museum,Pizza Place,Castle,Museum
24,Liverpool,121.8,39.4,37.7,131051.0,0.683695,0.639778,44.059184,4,Coffee Shop,Hotel,Bar,Café,Burger Joint,Cocktail Bar,Pizza Place,Fast Food Restaurant,Italian Restaurant,Spanish Restaurant
27,Newcastle upon Tyne,122.6,36.0,43.2,160730.0,0.667205,0.642500,35.100000,4,Hotel,Pub,Restaurant,Bar,Indian Restaurant,Coffee Shop,Café,Art Gallery,Gastropub,Butcher
29,Nottingham,129.5,39.8,46.6,151731.0,0.639210,0.668883,46.951064,4,Pub,Bar,Coffee Shop,Café,Indian Restaurant,Bookstore,Pizza Place,Cocktail Bar,Nightclub,English Restaurant


In [60]:
# Analyse Cluster 5 numeric data
cluster_5 = cities_merged.loc[cities_merged['Cluster Labels'] == 4, cities_merged.columns[[0] + list(range(1, cities_merged.shape[1]))]]
cluster_5.describe(exclude=[object])

,Total crime,Violent crime,Theft offences,Property_Jan_2020,Employment rate,"(MeetExp, mean)","(ATT8SCR, mean)",Cluster Labels
count,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.0
mean,115.733333,41.406667,36.553333,137870.800000,0.713798,0.654209,43.888593,4.0
std,21.440804,7.308944,8.127806,19842.520862,0.055149,0.023894,4.243339,0.0
min,82.900000,31.500000,22.500000,100643.000000,0.627727,0.613125,35.100000,4.0
25%,100.550000,35.850000,32.050000,123135.000000,0.677823,0.641139,42.161250,4.0
50%,111.100000,39.800000,35.500000,136912.000000,0.716120,0.656275,44.192000,4.0
75%,126.050000,44.300000,41.000000,154803.000000,0.745155,0.667258,47.110949,4.0
max,158.000000,55.400000,51.300000,160730.000000,0.812950,0.708444,51.050000,4.0
